In [17]:
!pip install pandas

In [24]:
import pandas as pd
import numpy as np
import bs4
import sys
import requests

In [29]:
def get_list_of_university_towns():
    with open(r'C:\Users\ABHISHEK SHARMA\Downloads\University_towns (1).txt', 'r') as reader:
        lines= reader.readlines()
    reader.close()
    
    unidata= pd.DataFrame(columns=('State', 'RegionName'))
    count= 0
    state= ""
    region= ""
    
    for line in lines:
        line= line.rstrip('n')
        line= line.strip()
        if 'edit' in line:
            state= line.split('[')[0]
        else:
            region=line.split('(',1)[0]
            unidata.loc[count]= [state,region]
            count= count+ 1
            
    return unidata

get_list_of_university_towns()

,State,RegionName
0,Alabama,AuburnÂ
1,Alabama,DothanÂ
2,Alabama,FairfieldÂ
3,Alabama,FlorenceÂ
4,Alabama,HomewoodÂ
...,...,...
828,Wisconsin,WhitewaterÂ
829,Wyoming,LaramieÂ
830,Puerto Rico,"MayagÃ¼ez, Puerto RicoÂ"
831,Puerto Rico,"Ponce, Puerto Rico"


In [36]:
df= pd.read_excel(r'C:\Users\ABHISHEK SHARMA\Downloads\gdplev.xlsx',skiprows=219)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,1999Q4,9899.4,12877.6,Unnamed: 7
0,NaN,NaN,NaN,NaN,2000Q1,10002.9,12924.2,NaN
1,NaN,NaN,NaN,NaN,2000Q2,10247.7,13160.8,NaN
2,NaN,NaN,NaN,NaN,2000Q3,10319.8,13178.4,NaN
3,NaN,NaN,NaN,NaN,2000Q4,10439.0,13260.5,NaN
4,NaN,NaN,NaN,NaN,2001Q1,10472.9,13222.7,NaN
...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,2020Q1,21561.1,19010.8,NaN
81,NaN,NaN,NaN,NaN,2020Q2,19520.1,17302.5,NaN
82,NaN,NaN,NaN,NaN,2020Q3,21170.3,18596.5,NaN
83,NaN,NaN,NaN,NaN,2020Q4,21494.7,18794.4,NaN


In [37]:
def clean_GDP_data():
    df= pd.read_excel(r'C:\Users\ABHISHEK SHARMA\Downloads\gdplev.xlsx',skiprows=219)
    df= df.rename(columns={'1999Q4':'Quarter',12877.6:'GDP'})
    df= df[['Quarter','GDP']]
    return df
clean_GDP_data().head()

    
    

,Quarter,GDP
0,2000Q1,12924.2
1,2000Q2,13160.8
2,2000Q3,13178.4
3,2000Q4,13260.5
4,2001Q1,13222.7


In [40]:
def recession_start():
    df= clean_GDP_data()
    i=2
    length= len(df['GDP'])
    recession_start= ''
    while i< length:
        if df.GDP[i] < df.GDP[i-1] and df.GDP[i-1] < df.GDP[i-2]:
            recession_start= df.Quarter[i-1]
            break 
        i+=1
    return recession_start
recession_start()

'2008Q3'

In [49]:
def get_recession_bottom():
    df= pd.read_excel(r'C:\Users\ABHISHEK SHARMA\Downloads\gdplev.xlsx',skiprows=219)
    df= df.rename(columns={'1999Q4':'Quarter',12877.6:'GDP'})
    df= df[['Quarter','GDP']]
                  
    i=2
    length= len(df['GDP'])
    in_recession= False
    recession_bottom=''
    lowest_gdp= 1000000000
    while i< length:
        if df.GDP[i] < df.GDP[i-1] and df.GDP[i-1] < df.GDP[i-2]:
                  in_recession= True
        if in_recession== True and df.GDP[i] < lowest_gdp:
                lowest_gdp= df.GDP[i]
                recession_bottom= df.Quarter[i]
        if in_recession== True and df.GDP[i] > df.GDP[i-1] and df.GDP[i-1] > df.GDP[i-2]:
                  in_recession= False
        i+=1
    return recession_bottom
get_recession_bottom()                  

'2009Q2'

## Convert house data to quarters

In [69]:
def convert_house_data_to_quarters():
    states= {
        'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AS':'American Samoa','AZ':'Arizona','CA':'California','CO':'Colorado','CT':'Connecticut','DC':'District of Columbia','DE':'Delaware','FL':'Florida','GA'
             :'Georgia','GU':'Guam','HI':'Hawaii','IA':'Iowa','ID':'Idaho','IL':'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana','MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri','MP':'Northern Mariana Islands','MS':'Mississippi','MT':'Montana','NA':'National','NC':'North Carolina'
    }
    
    house= pd.read_csv(r'C:\Users\ABHISHEK SHARMA\Downloads\City_Zhvi_AllHomes.csv')
    house['State'].replace(states,inplace= True)
    house= house.set_index(["State","RegionName"])
    house= house.iloc[:,49:250]
    
    def quarters(col):
        if col.endswith(("01","02","03")):
            s= col[:4]+ "q1"
        elif col.endswith(("04","05","06")):
            s= col[:4]+ "q2"
        elif col.endswith(("07","08","09")):
            s= col[:4]+ "q3"
        else:
            s= col[:4]+ "q4"
        return s
    
    print(house)
    #house= house.groupby(quarters,axis=1).mean()
    #print(house.groupby(quarters,axis=1).mean())
    #house= house.sort_index()
    return house
                
        

convert_house_data_to_quarters()     

                         1999-08-31  1999-09-30  1999-10-31  1999-11-30  \
State       RegionName                                                    
NY          New York       224705.0    226911.0    228704.0    230438.0   
California  Los Angeles    227833.0    228984.0    230138.0    231957.0   
TX          Houston        105728.0    106193.0    106598.0    107025.0   
Illinois    Chicago        161674.0    162512.0    163387.0    164519.0   
TX          San Antonio    102525.0    102869.0    103193.0    103574.0   
...                             ...         ...         ...         ...   
Minnesota   Winton              NaN         NaN         NaN         NaN   
Mississippi Eastabuchie         NaN         NaN         NaN         NaN   
TX          Dean                NaN         NaN         NaN         NaN   
Georgia     Pulaski             NaN         NaN         NaN         NaN   
PA          New Paris           NaN         NaN         NaN         NaN   

                        

,,1999-08-31,1999-09-30,1999-10-31,1999-11-30,1999-12-31,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,224705.0,226911.0,228704.0,230438.0,232207.0,234131.0,236741.0,239712.0,242688.0,245468.0,...,515516.0,518826.0,521580.0,524595.0,528104.0,531372.0,534094.0,536566.0,538825.0,541981.0
California,Los Angeles,227833.0,228984.0,230138.0,231957.0,233760.0,235520.0,237065.0,239037.0,241083.0,243424.0,...,566876.0,569628.0,572582.0,577344.0,581154.0,586803.0,589990.0,593866.0,596180.0,602443.0
TX,Houston,105728.0,106193.0,106598.0,107025.0,107515.0,107941.0,108083.0,108027.0,108068.0,108085.0,...,160485.0,160989.0,161435.0,162239.0,162693.0,163195.0,164085.0,165026.0,165626.0,165695.0
Illinois,Chicago,161674.0,162512.0,163387.0,164519.0,165858.0,167642.0,169335.0,171126.0,172721.0,174434.0,...,226575.0,227390.0,228045.0,228285.0,228738.0,229794.0,231084.0,233137.0,234677.0,235835.0
TX,San Antonio,102525.0,102869.0,103193.0,103574.0,103949.0,104227.0,104412.0,104482.0,104524.0,103866.0,...,147433.0,148906.0,149977.0,151001.0,151516.0,152312.0,153074.0,153465.0,154122.0,155197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Minnesota,Winton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70932.0,71578.0,71566.0,72035.0,70837.0,69376.0,68674.0,70155.0,71401.0,71007.0
Mississippi,Eastabuchie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61003.0,61503.0,61988.0,62413.0,62001.0,61484.0,62126.0,63003.0,64294.0,64483.0
TX,Dean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,142794.0,143746.0,144391.0,144496.0,144022.0,143835.0,144340.0,145625.0,146608.0,147414.0


In [66]:
unitowns= get_list_of_university_towns()
unitowns= unitowns.set_index(['State','RegionName'])
housingdata= convert_house_data_to_quarters() 

housingdata['2008q2']= housingdata['2008q2']/housingdata['2009q2']
housingdata= housingdata['2008q2'].to_frame()
housingdata= housingdata.dropna()

KeyError: '2008q2'